In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "quantium-customer-dataset"

## Introduction

## Business Task

## Loading Packages

In [2]:
library(tidyverse)
library(lubridate)
library(dplyr)
library(ggplot2)
library(tidyr)
library(skimr)
library(here)
library(janitor)
library(readxl)
library(data.table)
library(stringr)

here() starts at /kaggle/working


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Importing Dataset
Dataset used in this project is from Quantium

In [3]:
purchase_behaviour <- read.csv("/kaggle/input/quantium-customer-dataset/QVI_purchase_behaviour.csv")
transaction <- read_excel("/kaggle/input/quantium-customer-dataset/QVI_transaction_data.xlsx")

In [4]:
head(purchase_behaviour)
head(transaction)

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
,<int>,<chr>,<chr>
1,1000,YOUNG SINGLES/COUPLES,Premium
2,1002,YOUNG SINGLES/COUPLES,Mainstream
3,1003,YOUNG FAMILIES,Budget
4,1004,OLDER SINGLES/COUPLES,Mainstream
5,1005,MIDAGE SINGLES/COUPLES,Mainstream
6,1007,YOUNG SINGLES/COUPLES,Budget


DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8
43604,4,4074,2982,57,Old El Paso Salsa Dip Tomato Mild 300g,1,5.1


## Examining Transaction Data

In [5]:
transaction_df <- clean_names(transaction)
skim_without_charts(transaction_df)

#### Convert date column to a date format

In [ ]:
transaction_df$date <- as.Date(transaction_df$date, origin = "1899-12-30")
head(transaction_df)

#### Examine the words in prode_name to see if there are any incorrect entries such as products that are not chips

In [ ]:
product_words <- data.table(unlist(strsplit(unique(transaction_df$prod_name), " ")))
print(product_words)

####  Remove digits, and special characters, and then sort the distinct words by frequency of occurrence

In [ ]:
##### Remove characters

words_data <- str_replace_all(product_words, "[^[:alnum:]]", " ")
words_data

In [ ]:
#### Remove digits
words_clean <- gsub('[[:digit:]]+', '',words_data)
words_clean 

In [ ]:
#### Make a table
words_product <- data.table(unlist(strsplit(unique(words_clean)," ")))
setnames(words_product, "words")
words_product

#### Look at the most common words by counting the number of times a word appears and sorting them by this frequency in order of highest to lowest frequency

In [ ]:
#### Remove blank, count, and sort

words_product %>%
mutate(words = na_if(words, "")) %>% 
    filter(!is.na(words)) %>%
    group_by(words) %>%
    count(words, sort= TRUE)
    

There are salsa products in the dataset
#### Remove SALSA product

In [ ]:
#### create salsa phrase
remove_salsa <- c('salsa', 'Salsa','SALSA')
#### remove rows than contain salsa on transaction dataset
clean_transaction <- transaction_df[ !grepl(paste(remove_salsa, collapse="|"), transaction_df$prod_name),]

#### Summarise the data to check for nulls and possible outliers

In [ ]:
summary(clean_transaction)

There are no nulls in the columns but product quantity appears to have an outlier which we should investigate further. Let's investigate further the case where 200 packets of chips are bought in one transaction.

#### Filter the dataset to find the outlier

In [ ]:
clean_transaction %>% group_by(prod_name) %>% filter(prod_qty == 200)

There are two transactions where 200 packets of chips are bought in one transaction and both of these transactions were by the same customer

#### see if the customer has had other transactions

In [ ]:
clean_transaction %>% filter (lylty_card_nbr==226000)

It looks like this customer (226000) has only had the two transactions over the year and is not an ordinary retail customer. The customer might be buying chips for commercial purposes instead. We'll remove this loyalty card number from further analysis

#### Filter out the customer based on the loyalty card number

In [ ]:
new_transaction <- clean_transaction %>% filter(lylty_card_nbr!=226000)

Look at the number of transaction lines over time to see if there are any obvious data issues such as missing data
#### Count the number of transactions by date

In [ ]:
new_transaction %>% count(date)

There's only 364 rows, meaning only 364 dates which indicates a missing date. Let's create a sequence of dates from 1 Jul 2018 to 30 Jun 2019 and use this to create a chart of number of transactions over time to find the missing date

#### Create a sequence of dates and join this the count of transactions by date
create a column of dates that includes every day from 1 Jul 2018 to 30 Jun 2019, and join it onto the data to fill in the missing day

In [ ]:
#### Sequence of date
all_dates <- data.frame(date = seq(as.Date("2018-07-01"), as.Date("2019-06-30"), by = "day"))

In [ ]:
#### Join squence of date and new_transaction date
transaction_by_day <- all_dates %>%
  left_join(new_transaction, by = "date") %>% count(date)
transaction_by_day

#### Setting plot themes to format graphs

In [ ]:
theme_set(theme_bw())
theme_update(plot.title = element_text(hjust = 0.5))

#### Plot transactions over time

In [ ]:
ggplot(transaction_by_day, aes(x = date, y = n)) +
geom_line() +
labs(x = "Day", y = "Number of transactions", title = "Transactions over time") +
scale_x_date(breaks = "1 month") +  theme(axis.text.x = element_text(angle = 90, vjust = 0.5))

 There is an increase in purchases in December and a break in late December.

#### Filter to December and look at individual days

In [ ]:
december_data <- subset(transaction_by_day, format(date, "%m") == "12")
december_data

In [ ]:
ggplot(december_data, aes(x = date, y = n)) +
geom_line() +
labs(x = "Day", y = "Number of transactions", title = "Transactions over time") +
scale_x_date(breaks = "1 day") +  theme(axis.text.x = element_text(angle = 90, vjust = 0.5))

We can see that the increase in sales occurs in the lead-up to Christmas and that there are zero sales on Christmas day itself. This is due to shops being closed on Christmas day.

Now that we are satisfied that the data no longer has outliers, we can move on to creating other features such as brand of chips or pack size from PROD_NAME. We will start with pack size.

#### Pack size 
We can work this out by taking the digits that are in prod_name

In [ ]:
setDT(new_transaction)
new_transaction[, pack_size := parse_number(prod_name)]
new_transaction

 Let's check if the pack sizes look sensible

In [ ]:
transaction_pack_size <- new_transaction[, .N,pack_size][order(pack_size)]
transaction_pack_size

The largest size is 380g and the smallest size is 70g - seems sensible!

#### Plot a histogram showing the number of transactions by pack size

In [ ]:
ggplot(transaction_pack_size, aes(x = pack_size)) +
geom_histogram(binwidth=50,color = 'white',fill ='blue') +
labs(x = "Pack Size", y = "Number of transactions", title = "Transactions by Pack Size")+
theme_minimal()

 #### Create brands
 We can use the first word in prode_name to work out the brand name
 ##### Create a column which contains the brand of the product, by extracting it from the product name

In [ ]:
brand_transaction <- new_transaction %>%
  mutate(brand = str_extract(prod_name, "^[^\\d]+"))
brand_transaction %>% group_by(brand)

Some of the brand names look like they are of the same brands - such as RED and RRD, which are both Red Rock Deli chips

#### Clean brand names

In [ ]:
clean_transaction <- brand_transaction %>%
    mutate(brand = case_when(brand == "RED" ~ "RRD",TRUE ~ brand))
clean_transaction

## Examining customer data

In [ ]:
purchase_df <- clean_names(purchase_behaviour)
skim_without_charts(purchase_df)

#### Merge transaction data to customer data

In [ ]:
all_data <- merge(clean_transaction, purchase_df, all.x = TRUE)
all_data

As the number of rows in `all_data` is the same as that of `clean_transaction`, we can be sure that no duplicates were created. This is because we created `all_data` by setting `all.x = TRUE` (in other words, a left join) which means take all the rows in `clean_transaction` and find rows with matching values in shared columns and then joining the details in these rows to the `x` or the first mentioned table.

#### See if any transactions did not have a matched customer

In [ ]:
skim_without_charts(all_data)

There are no nulls. So all our customers in the transaction data has been accounted for in the customer dataset

In [ ]:
write.csv(all_data, "all_data.csv",row.names = FALSE)

Data exploration is now complete